In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn.metrics import recall_score,precision_score,accuracy_score,classification_report,confusion_matrix,f1_score
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

In [ ]:
df2 = pd.read_csv('/content/dataset_with_company_count_M (4).csv', sep=';')

In [ ]:
df2

,vacancy_type,area_orig,age,gender.id,education.level.id,salary.amount,total_experience.months,month,year,inflation_rate,...,natural_prirost_rozhd,mortality,median_salary,percent_with_internet,migration_prirost,number_top_school,prirost_naselenia,percent_poor,rate_of_quality_life,exp_company_count
0,Инсталлятор,Абинск,34.0,male,2.0,90000.0,90.0,1,2025,9.92,...,-0.5,12.5,33903.0,68.3,62.54,0.000000,10.31,11.6,76.130,1
1,Инсталлятор,Абинск,32.0,male,2.0,30000.0,47.0,1,2025,9.92,...,-0.5,12.5,33903.0,68.3,62.54,0.000000,10.31,11.6,76.130,2
2,Менеджер по поддержке продаж,Абинск,41.0,female,5.0,70000.0,199.0,1,2025,9.92,...,-0.5,12.5,33903.0,68.3,62.54,0.000000,10.31,11.6,76.130,6
3,Инсталлятор,Азов,50.0,male,2.0,80000.0,355.0,1,2025,9.92,...,-3.1,13.4,32241.0,68.5,4.51,0.418252,-1.10,13.9,68.296,1
4,Инсталлятор,Азов,24.0,male,2.0,70000.0,55.0,12,2024,9.51,...,-3.1,13.4,32241.0,68.5,4.51,0.418252,-1.10,13.9,68.296,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63458,Агент ГАП,Воронеж,35.0,female,5.0,70000.0,144.0,1,2025,9.92,...,-5.0,14.6,32989.0,57.7,43.31,0.763397,0.83,9.3,66.065,5
63459,Менеджер по сохранению клиентов,Воронеж,19.0,male,2.0,50000.0,12.0,1,2025,9.92,...,-5.0,14.6,32989.0,57.7,43.31,0.763397,0.83,9.3,66.065,2
63460,Агент ТМ,Воронеж,43.0,male,5.0,60000.0,108.0,1,2025,9.92,...,-5.0,14.6,32989.0,57.7,43.31,0.763397,0.83,9.3,66.065,1
63461,Агент ГАП,Ярославль,36.0,female,6.0,90000.0,137.0,1,2025,9.92,...,-4.7,15.2,34175.0,63.8,7.14,1.437046,-0.93,10.9,59.367,4


In [ ]:
! pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 17.9 MB/s eta 0:00:00


In [ ]:
df2 = df2[(df2['salary.amount'] >= 29000) & (df2['salary.amount'] <= 91000)]


In [ ]:
df2=df2.drop(columns=['area_orig', 'year', 'month'])

In [ ]:
df2=df2.dropna()

In [ ]:
df2=df2.drop_duplicates()

In [ ]:
df2.shape

(58333, 107)

In [ ]:
df2['appart_price_pervichniy_market'] = pd.to_numeric(df2['appart_price_pervichniy_market'], errors='coerce')


In [ ]:
TARGET = 'salary.amount'
X = df2.drop(columns=[TARGET])
y = df2[TARGET].astype(float)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
df2

,vacancy_type,age,gender.id,education.level.id,salary.amount,total_experience.months,inflation_rate,interest_rate,Region Name,avg_salary_per_region,...,relation_profit_prozit_minimum,natural_prirost_rozhd,mortality,median_salary,percent_with_internet,migration_prirost,number_top_school,prirost_naselenia,percent_poor,rate_of_quality_life
0,Агент ГАП,19.0,female,2.0,40000.0,5.0,9.92,21,Забайкальский край,87620.0,...,1.69,1.7,11.7,39553.0,63.6,-74.12,0.000000,-3.73,21.4,36.372
1,Агент ТМ,20.0,female,2.0,30000.0,34.0,9.92,21,Оренбургская область,64598.0,...,2.08,-1.7,13.3,32348.0,56.8,-42.74,0.000000,-2.60,14.5,56.789
2,Агент ГАП,32.0,female,1.0,35000.0,83.0,9.92,21,Забайкальский край,87620.0,...,1.69,1.7,11.7,39553.0,63.6,-74.12,0.000000,-3.73,21.4,36.372
3,Агент ГАП,22.0,female,2.0,60000.0,36.0,8.88,21,Саратовская область,58513.0,...,1.97,-4.1,13.6,30113.0,61.8,-24.93,1.434777,-3.33,17.4,55.762
4,Агент ГАП,42.0,female,5.0,60000.0,239.0,8.88,21,Саратовская область,58513.0,...,1.97,-4.1,13.6,30113.0,61.8,-24.93,1.434777,-3.33,17.4,55.762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59685,Менеджер по поддержке продаж,26.0,male,2.0,70000.0,98.0,9.51,21,Хабаровский край,116612.0,...,2.26,-1.0,13.0,50197.0,75.0,-27.73,0.000000,-0.94,12.5,57.681
59686,Агент ГАП,29.0,male,5.0,40000.0,93.0,8.88,21,Саратовская область,58513.0,...,1.97,-4.1,13.6,30113.0,61.8,-24.93,1.434777,-3.33,17.4,55.762
59687,Агент ГАП,29.0,female,2.0,50000.0,11.0,9.92,21,Забайкальский край,87620.0,...,1.69,1.7,11.7,39553.0,63.6,-74.12,0.000000,-3.73,21.4,36.372
59688,Агент ГАП,28.0,male,3.0,55000.0,80.0,9.92,21,Иркутская область,87473.0,...,1.82,0.5,12.9,44371.0,62.8,-24.63,1.480879,-1.62,20.0,49.924


In [ ]:
! pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.3 MB/s eta 0:00:00


In [ ]:
df2 = pd.get_dummies(df2, columns=['vacancy_type', 'gender.id','Region Name'
])

In [ ]:
! pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.2 MB/s eta 0:00:00


In [ ]:
df2.columns = [f'c{i+1}' for i in range(len(df2.columns))]


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, VotingRegressor, StackingRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.base import clone

def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-8))) * 100

def prepare_data(df, target_col='salary.amount', test_size=0.2, random_state=42):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    numeric_cols = X.select_dtypes(include=np.number).columns
    scaler = StandardScaler()
    X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
    X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

    return X_train, X_test, y_train, y_test, scaler

def build_base_models(random_state=42):
    gb_model = GradientBoostingRegressor(
        n_estimators=181,
        learning_rate=0.20214862522004154,
        max_depth=4,
        min_samples_split=9,
        min_samples_leaf=2,
        subsample=0.9212026339680122,
        max_features=None,
        random_state=random_state
    )

    rf_model = RandomForestRegressor(
        max_depth=13,
        max_features=None,
        min_samples_leaf=3,
        min_samples_split=18,
        n_estimators=222,
        n_jobs=-1,
        random_state=random_state
    )

    lgbm_model = LGBMRegressor(
        num_leaves=245,
        max_depth=4,
        learning_rate=0.10368980231785788,
        n_estimators=264,
        min_child_samples=13,
        subsample=0.8556391975919111,
        colsample_bytree=0.7811969309029951,
        reg_alpha=0.6901682685174545,
        reg_lambda=0.9852504970380468,
        random_state=random_state,
        n_jobs=-1
    )

    cat_model = CatBoostRegressor(
        iterations=648,
        learning_rate=0.10578575537254957,
        depth=6,
        l2_leaf_reg=4.655292971188782,
        random_strength=0.8510155082185262,
        silent=True,
        random_seed=random_state
    )

    return {
        'GradientBoosting': gb_model,
        'RandomForest': rf_model,
        'LightGBM': lgbm_model,
        'CatBoost': cat_model
    }

def evaluate_model(model, X_test, y_test, model_name):
    """Оценка модели и вывод метрик"""
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = calculate_mape(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(f"{model_name} Results:")
    print(f"R2 Score: {r2:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MAPE: {mape:.4f}%")
    print(f"RMSE: {rmse:.4f}\n")

    return {
        'R2': r2,
        'MAE': mae,
        'MAPE': mape,
        'RMSE': rmse
    }

def build_ensembles(X_train, y_train, base_models):
    voting = VotingRegressor(
        estimators=[(name, clone(model)) for name, model in base_models.items()],
        n_jobs=-1
    )
    voting.fit(X_train, y_train)

    stacking = StackingRegressor(
        estimators=[(name, clone(model)) for name, model in base_models.items()],
        final_estimator=LinearRegression(),
        cv=5,
        n_jobs=-1,
        passthrough=True
    )
    stacking.fit(X_train, y_train)

    return voting, stacking

def run_ensembles(df, target_col='c3'):
    X_train, X_test, y_train, y_test, scaler = prepare_data(df, target_col)

    base_models = build_base_models()
    print("=== Base Models Evaluation ===")
    base_results = {}
    for name, model in base_models.items():
        model.fit(X_train, y_train)
        metrics = evaluate_model(model, X_test, y_test, name)
        base_results[name] = metrics

    voting, stacking = build_ensembles(X_train, y_train, base_models)
    print("\n=== Ensembles Evaluation ===")
    voting_metrics = evaluate_model(voting, X_test, y_test, "Voting Ensemble")
    stacking_metrics = evaluate_model(stacking, X_test, y_test, "Stacking Ensemble")

    results = {
        'Base Models': base_results,
        'Voting Ensemble': voting_metrics,
        'Stacking Ensemble': stacking_metrics
    }

    metrics_df = pd.DataFrame(
        [(model, vals['R2'], vals['MAE'], vals['MAPE'], vals['RMSE'])
         for model, vals in results.items()
         for vals in [vals] if isinstance(vals, dict)],
        columns=['Model', 'R2', 'MAE', 'MAPE (%)', 'RMSE']
    )

    base_metrics_df = pd.DataFrame(
        [(name, vals['R2'], vals['MAE'], vals['MAPE'], vals['RMSE'])
         for name, vals in base_results.items()],
        columns=['Model', 'R2', 'MAE', 'MAPE (%)', 'RMSE']
    )

    print("\n=== Summary Metrics ===")
    print("\nBase Models:")
    print(base_metrics_df.to_markdown(index=False))

    print("\nEnsembles:")
    print(metrics_df[metrics_df['Model'].isin(['Voting Ensemble', 'Stacking Ensemble'])].to_markdown(index=False))

    import matplotlib.pyplot as plt
    import seaborn as sns

    plt.figure(figsize=(15, 10))
    for i, (name, model) in enumerate(base_models.items(), 1):
        plt.subplot(2, 2, i)
        if hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
            indices = np.argsort(importances)[-10:]
            plt.title(f'{name} Feature Importance')
            plt.barh(range(len(indices)), importances[indices], color='b', align='center')
            plt.yticks(range(len(indices)), [X_train.columns[i] for i in indices])
            plt.xlabel('Relative Importance')

    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(15, 5))

    best_base_name = max(base_results.items(), key=lambda x: x[1]['R2'])[0]
    best_base_model = base_models[best_base_name]
    y_pred_base = best_base_model.predict(X_test)

    y_pred_stack = stacking.predict(X_test)

    plt.subplot(1, 2, 1)
    plt.scatter(y_test, y_pred_base, alpha=0.3, label=best_base_name)
    plt.scatter(y_test, y_pred_stack, alpha=0.3, label='Stacking Ensemble')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title('Actual vs Predicted Values')
    plt.legend()

    plt.subplot(1, 2, 2)
    residuals_base = y_test - y_pred_base
    residuals_stack = y_test - y_pred_stack
    plt.scatter(y_pred_base, residuals_base, alpha=0.3, label=best_base_name)
    plt.scatter(y_pred_stack, residuals_stack, alpha=0.3, label='Stacking Ensemble')
    plt.hlines(y=0, xmin=y_pred_base.min(), xmax=y_pred_base.max(), colors='k', linestyles='dashed')
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.title('Residual Analysis')
    plt.legend()

    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 6))
    metrics_to_compare = ['R2', 'MAE', 'RMSE']
    for metric in metrics_to_compare:
        values = [base_results[name][metric] for name in base_models]
        values.extend([voting_metrics[metric], stacking_metrics[metric]])
        plt.plot(values, marker='o', label=metric)

    plt.xticks(range(len(base_models)+2),
               list(base_models.keys()) + ['Voting', 'Stacking'],
               rotation=45)
    plt.title('Model Comparison Across Metrics')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)
    plt.show()

    return results, {
        'voting': voting,
        'stacking': stacking,
        'base_models': base_models,
        'scaler': scaler
    }, metrics_df

results, models, metrics_df = run_ensembles(df2)

=== Base Models Evaluation ===
GradientBoosting Results:
R2 Score: 0.2488
MAE: 11809.5215
MAPE: 22.5363%
RMSE: 14446.7033

RandomForest Results:
R2 Score: 0.2410
MAE: 11874.0576
MAPE: 22.6976%
RMSE: 14522.0680

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012390 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1764
[LightGBM] [Info] Number of data points in the train set: 46666, number of used features: 101
[LightGBM] [Info] Start training from score 59076.541529
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(



=== Ensembles Evaluation ===
Voting Ensemble Results:
R2 Score: 0.2549
MAE: 11765.6470
MAPE: 22.4708%
RMSE: 14388.5588

Stacking Ensemble Results:
R2 Score: 0.2558
MAE: 11760.4521
MAPE: 22.4630%
RMSE: 14379.9186



KeyError: 'R2'